In [102]:
lhsW={"qv":[1,[("sqv","qv1")]]}
rhsW={"P":[-1,[("s_P1","P1")]],"E":[1,[("s_E1","E1")]]}
L=2.5e6 #vaporization 
lhsE={"k":[1,[("sk1","k1")]],"s":[1,[("ss1","s1")]],"qv":[L,[("sqv","qv1")]]}
rhsE={"E":[L,[("s_E1","E1")]],"H":[1,[("s_H1","H1")]],"RS":[-1,[("s_rs","RS1")]]\
                                  ,"RT":[1,[("s_rt","RT1")]]}
class budEquation:
    def __init__(self,lhs,rhs):
        self.lhs=lhs
        self.rhs=rhs
        
watBudget=budEquation(lhsW,rhsW)
enBudget=budEquation(lhsE,rhsE)
cEq=[watBudget,enBudget]
def lagrangeEquations(cEq):
    varHF={}
    varVF={}
    eqVF={}
    eqHF={}
    eqVF_x={}
    eqVF_y={}
    for (eq,i) in zip(cEq,range(len(cEq))):
        for item in eq.lhs.keys():
            #print(item)
            if item not in varHF.keys():
                #print(eq.lhs[item])
                varHF[item]=eq.lhs[item][1]
                uncert_def=eq.lhs[item][1]
                eqStX=''
                eqStY=''
                #print(uncert_def)
                for term in uncert_def:
                    if eqStX=='':
                        eqStX="-("+item+"_x-"+term[1]+"_x)/"+term[0]+"^2"
                    else:
                        eqStX+="-("+item+"_x-"+term[1]+"_x)/"+term[0]+"^2"
                eqStX+="+"+str('(%6.2e)'%eq.lhs[item][0])+"*lambda_%i.gradX"%i
                eqVF_x[item]=eqStX
                for term in uncert_def:
                    if eqStY=='':
                        eqStY="-("+item+"_y-"+term[1]+"_y)/"+term[0]+"^2"
                    else:
                        eqStY+="-("+item+"_y-"+term[1]+"_y)/"+term[0]+"^2"
                eqStY+="+"+str('(%6.2e)'%eq.lhs[item][0])+"*lambda_%i.gradY"%i
                eqVF_y[item]=eqStY
            else:
                eqStY=eqVF_y[item]
                eqStY+="+"+str('(%6.2e)'%eq.lhs[item][0])+"*lambda_%i.gradY"%i
                eqVF_y[item]=eqStY
                eqStX=eqVF_x[item]
                eqStX+="+"+str('(%6.2e)'%eq.lhs[item][0])+"*lambda_%i.gradX"%i
                eqVF_x[item]=eqStX
        for item in eq.rhs.keys():
            #print(item)
            if item not in varVF.keys():
                #print(eq.rhs[item])
                varVF[item]=eq.rhs[item][1]
                uncert_def=eq.rhs[item][1]
                eqSt=''
                for term in uncert_def:
                    if eqSt=='':
                        eqSt="("+item+"-"+term[1]+")/"+term[0]+"^2"
                    else:
                        eqSt+="+("+item+"-"+term[1]+")/"+term[0]+"^2"
                eqSt+="+"+str('(%6.2e)'%(-eq.rhs[item][0]))+"*lambda_%i"%i
                eqHF[item]=eqSt
            else:
                eqSt=eqHF[item]
                eqSt+="+"+str('(%6.2e)'%(-eq.rhs[item][0]))+"*lambda_%i"%i
                eqHF[item]=eqSt
    return varHF,varVF,eqHF,eqVF_x, eqVF_y

varHF,varVF,eqHF,eqVF_x,eqVF_y=lagrangeEquations(cEq)
for item in eqHF.keys():
    print("eq[%s]:=%s=0"%(item,eqHF[item]))
    
for item in eqVF_x.keys():
    print('-------------------------------')
    print("eqX[%s]:=%s=0 \neqY[%s]:=%s=0"%(item,eqVF_x[item],item,eqVF_y[item]))

for (eq,i) in zip(cEq,range(len(cEq))):
    lhs=[]
    rhs=[]
    print('--------------------------------')
    for (eqd,j) in zip(cEq,range(len(cEq))):
        coeff='('
        for item in eq.lhs.keys():
            term=eq.lhs[item]
            if item in eqd.lhs.keys():
                termj=eqd.lhs[item]
                if coeff=='(':
                    coeff=coeff+'%s^2*%6.2e'%(term[1][0][0],term[0]*termj[0])
                else:
                    coeff=coeff+'+%s^2*%6.2e'%(term[1][0][0],term[0]*termj[0])
        coeff=coeff+')*del^2(lambda_%i)'%j
        lhs.append(coeff)
    coeff='' 
    for (eqd,j) in zip(cEq,range(len(cEq))):
        coeff1='()'
        for item in eq.rhs.keys():
            term=eq.rhs[item]
            if item in eqd.rhs.keys():
                termj=eqd.rhs[item]
                if coeff=='':
                    coeff=coeff+'%s^2*(%6.2e)*lambda_%i'%(term[1][0][0],term[0]*termj[0],j)
                else:
                    coeff=coeff+'+%s^2*(%6.2e)*lambda_%i'%(term[1][0][0],term[0]*termj[0],j)
    rhs.append(coeff)
    print(lhs)
    print(rhs)

eq[P]:=(P-P1)/s_P1^2+(1.00e+00)*lambda_0=0
eq[E]:=(E-E1)/s_E1^2+(-1.00e+00)*lambda_0+(-2.50e+06)*lambda_1=0
eq[H]:=(H-H1)/s_H1^2+(-1.00e+00)*lambda_1=0
eq[RS]:=(RS-RS1)/s_rs^2+(1.00e+00)*lambda_1=0
eq[RT]:=(RT-RT1)/s_rt^2+(-1.00e+00)*lambda_1=0
-------------------------------
eqX[qv]:=-(qv_x-qv1_x)/sqv^2+(1.00e+00)*lambda_0.gradX+(2.50e+06)*lambda_1.gradX=0 
eqY[qv]:=-(qv_y-qv1_y)/sqv^2+(1.00e+00)*lambda_0.gradY+(2.50e+06)*lambda_1.gradY=0
-------------------------------
eqX[k]:=-(k_x-k1_x)/sk1^2+(1.00e+00)*lambda_1.gradX=0 
eqY[k]:=-(k_y-k1_y)/sk1^2+(1.00e+00)*lambda_1.gradY=0
-------------------------------
eqX[s]:=-(s_x-s1_x)/ss1^2+(1.00e+00)*lambda_1.gradX=0 
eqY[s]:=-(s_y-s1_y)/ss1^2+(1.00e+00)*lambda_1.gradY=0
--------------------------------
['(sqv^2*1.00e+00)*del^2(lambda_0)', '(sqv^2*2.50e+06)*del^2(lambda_1)']
['s_P1^2*(1.00e+00)*lambda_0+s_E1^2*(1.00e+00)*lambda_0+s_E1^2*(2.50e+06)*lambda_1']
--------------------------------
['(sqv^2*2.50e+06)*del^2(lambda_0)', '(sk1^2*1.00e